IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M3\NO_FAULT3


In [3]:
guasto=0

VOLO m3 NO FAULT

In [4]:
rcou_m3_nofault = pd.read_csv("RCOU.csv")
rcou_m3_nofault = rcou_m3_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m3_nofault = rcou_m3_nofault[((rcou_m3_nofault['C9'] >= 1350) & (rcou_m3_nofault['C10'] >= 1350) & (rcou_m3_nofault['C11'] >= 1350) & (rcou_m3_nofault['C12'] >= 1350) & (rcou_m3_nofault['C13'] >= 1350) & (rcou_m3_nofault['C14']>= 1350))]
rcou_m3_nofault=rcou_m3_nofault.reset_index(drop=True)
display(rcou_m3_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,85323839,1415,1364,1394,1387,1416,1363
1,85345385,1411,1395,1437,1368,1441,1363
2,85356318,1376,1441,1404,1415,1366,1450
3,85367015,1409,1423,1421,1411,1412,1420
4,85423541,1523,1364,1442,1456,1515,1375
...,...,...,...,...,...,...,...
6615,162729348,1551,1519,1559,1510,1447,1613
6616,162740384,1598,1461,1520,1545,1460,1599
6617,162751646,1620,1423,1544,1513,1512,1545
6618,162761906,1593,1437,1553,1485,1496,1543


In [5]:
min=rcou_m3_nofault['TimeUS'][0]
max=rcou_m3_nofault['TimeUS'][len(rcou_m3_nofault)-1]
print(max)
print(min)

162772643
85323839


In [6]:
xkf1_m3_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m3_nofault = xkf1_m3_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_nofault = xkf1_m3_nofault[((xkf1_m3_nofault['TimeUS'] >= min) & (xkf1_m3_nofault['TimeUS'] <= max))]
xkf1_m3_nofault = xkf1_m3_nofault.reset_index(drop=True)
print(xkf1_m3_nofault)

         TimeUS  Roll  Pitch     Yaw
0      85332681  1.98  -0.84    3.31
1      85343445  1.98  -0.88    3.31
2      85353408  1.91  -0.83    3.33
3      85364246  1.93  -0.80    3.33
4      85375369  2.02  -0.82    3.32
...         ...   ...    ...     ...
7005  162727101 -1.18   1.24  358.17
7006  162738085 -1.06   1.13  358.20
7007  162749622 -0.97   1.00  358.25
7008  162759649 -0.91   0.94  358.31
7009  162770128 -0.79   0.88  358.39

[7010 rows x 4 columns]


In [7]:
att_m3_nofault = pd.read_csv("ATT.csv")
att_m3_nofault = att_m3_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_nofault = att_m3_nofault[((att_m3_nofault['TimeUS'] >= min) & (att_m3_nofault['TimeUS'] <= max	))]
att_m3_nofault=att_m3_nofault.reset_index(drop=True)
print(att_m3_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      85334020     0.12  1.98      0.24  -0.84    3.31    3.31
1      85344873     0.12  1.98      0.25  -0.88    3.31    3.31
2      85355543     0.12  1.91      0.26  -0.83    3.31    3.33
3      85366178     0.12  1.93      0.26  -0.80    3.31    3.33
4      85377132     0.12  2.02      0.25  -0.82    3.31    3.32
...         ...      ...   ...       ...    ...     ...     ...
7007  162728615     0.96 -1.18     -0.36   1.24  359.91  358.18
7008  162739678     0.95 -1.06     -0.37   1.13  359.91  358.21
7009  162751006     0.93 -0.97     -0.38   1.00  359.91  358.26
7010  162761104     0.89 -0.91     -0.36   0.94  359.91  358.32
7011  162771993     0.83 -0.79     -0.33   0.88  359.91  358.40

[7012 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_nofault)>len(xkf1_m3_nofault):
    to_add=att_m3_nofault[len(xkf1_m3_nofault):]
    att_m3_nofault=att_m3_nofault[:len(xkf1_m3_nofault)]

if len(xkf1_m3_nofault)>len(att_m3_nofault):
    to_add=xkf1_m3_nofault[len(att_m3_nofault):]

for idx,i in enumerate(att_m3_nofault['Roll']):
    if(xkf1_m3_nofault['Roll'][idx] != i):
        att_m3_nofault['Roll'][idx] = ((att_m3_nofault['Roll'][idx] + xkf1_m3_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Pitch']):
    if(xkf1_m3_nofault['Pitch'][idx] != i):
        att_m3_nofault['Pitch'][idx] = ((att_m3_nofault['Pitch'][idx] + xkf1_m3_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Yaw']):
    if(xkf1_m3_nofault['Yaw'][idx] != i):
        att_m3_nofault['Yaw'][idx] = ((att_m3_nofault['Yaw'][idx] + xkf1_m3_nofault['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_nofault=pd.concat([att_m3_nofault,to_add])

print(att_m3_nofault)

C:\Users\chiar\AppData\Local\Temp\ipykernel_32000\1419942214.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_nofault['Roll'][idx] = ((att_m3_nofault['Roll'][idx] + xkf1_m3_nofault['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32000\1419942214.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_nofault['Pitch'][idx] = ((att_m3_nofault['Pitch'][idx] + xkf1_m3_nofault['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32000\1419942214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      85334020     0.12  1.980      0.24 -0.840    3.31    3.310
1      85344873     0.12  1.980      0.25 -0.880    3.31    3.310
2      85355543     0.12  1.910      0.26 -0.830    3.31    3.330
3      85366178     0.12  1.930      0.26 -0.800    3.31    3.330
4      85377132     0.12  2.020      0.25 -0.820    3.31    3.320
...         ...      ...    ...       ...    ...     ...      ...
7007  162728615     0.96 -1.075     -0.36  1.120  359.91  358.215
7008  162739678     0.95 -0.985     -0.37  1.035  359.91  358.260
7009  162751006     0.93 -0.880     -0.38  0.940  359.91  358.325
7010  162761104     0.89 -0.910     -0.36  0.940  359.91  358.320
7011  162771993     0.83 -0.790     -0.33  0.880  359.91  358.400

[7012 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_nofault = pd.read_csv("ESC_0.csv")
esc_0_m3_nofault = esc_0_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_nofault = esc_0_m3_nofault[((esc_0_m3_nofault['TimeUS'] >= min) & (esc_0_m3_nofault['TimeUS'] <= max))]
esc_0_m3_nofault=esc_0_m3_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_nofault = pd.read_csv("ESC_1.csv")
esc_1_m3_nofault = esc_1_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_nofault = esc_1_m3_nofault[((esc_1_m3_nofault['TimeUS'] >= min) & (esc_1_m3_nofault['TimeUS'] <= max))]
esc_1_m3_nofault=esc_1_m3_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_nofault = pd.read_csv("ESC_2.csv")
esc_2_m3_nofault = esc_2_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_nofault = esc_2_m3_nofault[((esc_2_m3_nofault['TimeUS'] >= min) & (esc_2_m3_nofault['TimeUS'] <= max))]
esc_2_m3_nofault=esc_2_m3_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_nofault = pd.read_csv("ESC_3.csv")
esc_3_m3_nofault = esc_3_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_nofault = esc_3_m3_nofault[((esc_3_m3_nofault['TimeUS'] >= min) & (esc_3_m3_nofault['TimeUS'] <= max))]
esc_3_m3_nofault=esc_3_m3_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_nofault = pd.read_csv("ESC_4.csv")
esc_4_m3_nofault = esc_4_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_nofault = esc_4_m3_nofault[((esc_4_m3_nofault['TimeUS'] >= min) & (esc_4_m3_nofault['TimeUS'] <= max))]
esc_4_m3_nofault=esc_4_m3_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_nofault = pd.read_csv("ESC_5.csv")
esc_5_m3_nofault = esc_5_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_nofault = esc_5_m3_nofault[((esc_5_m3_nofault['TimeUS'] >= min) & (esc_5_m3_nofault['TimeUS'] <= max))]
esc_5_m3_nofault=esc_5_m3_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_nofault = pd.read_csv("IMU_0.csv")
imu_0_m3_nofault = imu_0_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_nofault = imu_0_m3_nofault[((imu_0_m3_nofault['TimeUS'] >= min) & (imu_0_m3_nofault['TimeUS'] <= max))]

imu_1_m3_nofault = pd.read_csv("IMU_1.csv")
imu_1_m3_nofault = imu_1_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_nofault = imu_1_m3_nofault[((imu_1_m3_nofault['TimeUS'] >= min) & (imu_1_m3_nofault['TimeUS'] <= max))]

imu_2_m3_nofault = pd.read_csv("IMU_2.csv")
imu_2_m3_nofault = imu_2_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_nofault = imu_2_m3_nofault[((imu_2_m3_nofault['TimeUS'] >= min) & (imu_2_m3_nofault['TimeUS'] <= max))]

imu_m3_nofault = pd.concat((imu_0_m3_nofault, imu_1_m3_nofault, imu_2_m3_nofault))
imu_m3_nofault=imu_m3_nofault.groupby(imu_m3_nofault.TimeUS, as_index=False).mean()

display(imu_m3_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,85325609,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955
1,85328405,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685
2,85333916,-0.054220,-0.096859,-0.017186,-0.235180,-0.172751,-9.537430
3,85339625,0.009177,0.034897,0.022398,-0.238212,-0.308036,-10.156127
4,85342546,-0.040668,0.042636,0.027003,-0.262722,-0.363957,-10.009839
...,...,...,...,...,...,...,...
26643,162760938,0.072825,-0.033702,0.107191,1.234315,1.531393,-12.105240
26644,162763425,0.250181,-0.223798,0.084430,1.207410,1.538579,-12.511827
26645,162766175,0.185225,-0.168557,0.087139,0.760958,1.467667,-12.682097
26646,162768911,0.051801,0.000553,0.112503,0.974692,1.827939,-12.768413


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_nofault = pd.merge_ordered(imu_m3_nofault,att_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_nofault = pd.merge_ordered(m3_nofault, esc_0_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_1_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_2_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_3_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_4_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_5_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")
m3_nofault = pd.merge_ordered(m3_nofault, rcou_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m3_nofault["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

7012
7677
8323
8996
9646
10292
10939


In [13]:
m3_nofault = m3_nofault[~m3_nofault.TimeUS.isin(index_to_remove)]
m3_nofault["TimeUS"] = m3_nofault["TimeUS"] - m3_nofault.iloc[0]["TimeUS"]
m3_nofault["TimeUS"] = m3_nofault["TimeUS"].astype(int)
m3_nofault["Guasto"] = guasto

In [14]:
from datetime import timedelta
m3_nofault=m3_nofault.reset_index(drop=True)
m3_nofault['TimeUS'] = pd.to_datetime(m3_nofault['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0
1,00:00:00.002796,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0
2,00:00:00.008307,-0.054220,-0.096859,-0.017186,-0.235180,-0.172751,-9.537430,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0
3,00:00:00.014016,0.009177,0.034897,0.022398,-0.238212,-0.308036,-10.156127,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0
4,00:00:00.016937,-0.040668,0.042636,0.027003,-0.262722,-0.363957,-10.009839,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26643,00:01:17.435329,0.072825,-0.033702,0.107191,1.234315,1.531393,-12.105240,0.93,-0.88,-0.38,...,2.55,5545.0,2.77,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0,0
26644,00:01:17.437816,0.250181,-0.223798,0.084430,1.207410,1.538579,-12.511827,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0
26645,00:01:17.440566,0.185225,-0.168557,0.087139,0.760958,1.467667,-12.682097,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0
26646,00:01:17.443302,0.051801,0.000553,0.112503,0.974692,1.827939,-12.768413,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_nofault)/350)):
        V1=m3_nofault[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32000\3790083574.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_32000\3790083574.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.061821,0.013651,0.090033,0.132037,0.0,77.0,12.629197,5.300712,0.030588,...,7029.307607,3.519772,1.272937,-0.937632,3.695687,98.0,97.0,23.686251,17.224285,0
1,1.0,-0.061986,0.013622,0.096201,0.132004,0.0,78.0,12.646878,5.318549,0.030286,...,7162.506161,3.521903,1.270909,-0.930097,3.697445,92.0,91.0,23.719284,16.868575,0
2,2.0,-0.061997,0.013583,0.105105,0.131864,0.0,71.0,12.716850,5.376738,0.029827,...,7298.539973,3.524023,1.268884,-0.922543,3.699192,94.0,93.0,23.723576,16.461228,0
3,3.0,-0.061912,0.013547,0.111946,0.131688,0.0,88.0,12.819161,5.455327,0.029377,...,7437.234502,3.526130,1.266861,-0.914971,3.700929,94.0,93.0,23.699870,16.004476,0
4,4.0,-0.061910,0.013509,0.120737,0.131543,0.0,83.0,12.890680,5.491232,0.028968,...,7578.411760,3.528225,1.264841,-0.907380,3.702655,100.0,99.0,23.648949,15.500860,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71.0,-0.064962,0.011728,0.531863,0.126174,0.0,103.0,13.673290,5.426798,0.013470,...,13863.169960,3.635995,1.128534,-0.369251,3.787654,117.0,115.0,26.759850,16.109357,0
72,72.0,-0.064978,0.011700,0.540382,0.126074,0.0,107.0,13.669021,5.402995,0.013179,...,13588.104563,3.637825,1.127276,-0.361355,3.789246,115.0,113.0,27.188979,16.240753,0
73,73.0,-0.065116,0.011680,0.545898,0.126067,0.0,101.0,13.613273,5.321545,0.012936,...,13310.587647,3.639646,1.126018,-0.353450,3.790829,121.0,119.0,27.566091,16.353128,0
74,74.0,-0.065364,0.011679,0.543734,0.126190,0.0,101.0,13.509759,5.187101,0.012896,...,13031.031801,3.641459,1.124758,-0.345535,3.792405,115.0,113.0,27.891284,16.448760,0


In [18]:
path_file = path_file.replace(r"M3\NO_FAULT3", "")
os.chdir(path_file)
df_merged.to_csv('m3_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
